In [0]:
#!pip install tensorflow-gpu==2.0.0-alpha0
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])



In [0]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

In [0]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

In [0]:
trX

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [0]:
NUM_HIDDEN = 100
NUM_HIDDEN_2 = 100
NUM_HIDDEN_3 = 100

In [0]:
X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

In [0]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN_2, 4])
w_h1 = init_weights([NUM_HIDDEN, NUM_HIDDEN_2])
#w_h2 = init_weights([NUM_HIDDEN_2, NUM_HIDDEN_3])

In [0]:
def model(X, w_h, w_h1, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    h_1 = tf.nn.relu(tf.matmul(h, w_h1)) 
    #h_2 = tf.nn.relu(tf.matmul(h_1, w_h2))
    return tf.matmul(h_1, w_o)

In [0]:
py_x = model(X, w_h, w_h1, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = py_x,labels= Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

In [0]:
predict_op = tf.argmax(py_x, 1)

In [0]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [0]:
NUM_EPOCHS = 1000

In [0]:
BATCH_SIZE = 128

In [0]:
sess = tf.Session()
saver = tf.train.Saver()
with sess.as_default():
    tf.global_variables_initializer().run()
    
    for epoch in range(NUM_EPOCHS):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            
        accuracy = np.mean(np.argmax(trY, axis=1) ==
                           sess.run(predict_op, feed_dict={X: trX, Y: trY}))
        saved_path = saver.save(sess, './my-model', global_step=epoch)
        print(epoch, accuracy)

0 0.4171180931744312
1 0.4723726977248104
2 0.5102925243770314
3 0.5178764897074756
4 0.5297941495124594
5 0.5341278439869989
6 0.5341278439869989
7 0.5341278439869989
8 0.5341278439869989
9 0.5341278439869989
10 0.5341278439869989
11 0.5341278439869989
12 0.5341278439869989
13 0.5341278439869989
14 0.5341278439869989
15 0.5341278439869989
16 0.5341278439869989
17 0.5341278439869989
18 0.5341278439869989
19 0.5341278439869989
20 0.5341278439869989
21 0.5341278439869989
22 0.5341278439869989
23 0.5341278439869989
24 0.5341278439869989
25 0.5341278439869989
26 0.5341278439869989
27 0.5341278439869989
28 0.5341278439869989
29 0.5341278439869989
30 0.5341278439869989
31 0.5341278439869989
32 0.5341278439869989
33 0.5341278439869989
34 0.5341278439869989
35 0.5341278439869989
36 0.5341278439869989
37 0.5341278439869989
38 0.5341278439869989
39 0.5341278439869989
40 0.5341278439869989
41 0.5341278439869989
42 0.5341278439869989
43 0.5341278439869989
44 0.5341278439869989
45 0.534127843986998

In [0]:
numbers = np.arange(1, 101)
teX = np.transpose(binary_encode(numbers, NUM_DIGITS)) 

In [0]:
teY = sess.run(predict_op, feed_dict={X: teX})
output = np.vectorize(fizz_buzz)(numbers, teY)
print (output)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' 'fizz' 'fizz' '79' 'buzz' 'fizz'
 '82' '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz'
 '94' 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [0]:
actuals = [fizz_buzz(i, fizz_buzz_encode(i).argmax()) for i in numbers]

for i, (predicted, actual) in enumerate(zip(output, actuals)):
    if predicted != actual:
        print("{0} {1} {2}".format(i+1, predicted, actual))

77 fizz 77


In [0]:
Q = []
for i in range(1,101):
    if   i % 15 == 0: Q.append('fizzbuzz')
    elif i % 5  == 0: Q.append('buzz')
    elif i % 3  == 0: Q.append('fizz')
    else:             Q.append(str(i))

In [0]:
j= 0
for i in range(100):
  if Q[i]==output[i]:
    j = j+1

In [0]:
j

99

In [0]:
print(tf.__version__)

2.0.0-alpha0


In [0]:
print (w_h)

<tf.Variable 'Variable_4:0' shape=(10, 100) dtype=float32_ref>


In [0]:
py_x.save('aann.h5')

AttributeError: ignored

In [0]:
#saver = tf.train.Saver()

In [0]:
#from google.colab import files
#files.download("/tmp/model.ckpt")

FileNotFoundError: ignored

In [0]:

#from google.colab import files
#files.download('./my-model')


FileNotFoundError: ignored

In [0]:
#sess.save('my_model.h5')

AttributeError: ignored